<a href="https://colab.research.google.com/github/srirambandi/GAN/blob/master/wgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# when running in colab notebooks, first install library
!pip install import-ai
# upload respective dataset manually from examples directory of the library or download as below
!apt install subversion
!svn checkout https://github.com/srirambandi/ai/trunk/examples/MNIST

  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1
Suggested packages:
  db5.3-util libapache2-mod-svn subversion-tools
The following NEW packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1 subversion
0 upgraded, 5 newly installed, 0 to remove and 59 not upgraded.
Need to get 2,237 kB of archives.
After this operation, 9,910 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libapr1 amd64 1.6.3-2 [90.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libaprutil1 amd64 1.6.1-2 [84.4 kB]
Get:3 http://archive.ubuntu.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import ai
import numpy as np

In [4]:
z_dim = 100
gf_dim = 64
df_dim = 64

In [5]:
ai.manual_seed(2357)

In [6]:
def data_generator(m):
    train_dict = np.load('MNIST/train.npy', allow_pickle=True)
    test_dict = np.load('MNIST/test.npy', allow_pickle=True)
    data = np.concatenate([train_dict.item()['data'], test_dict.item()['data']])
    data = data.transpose(1, 2, 0)   # making data batch-last
    data = data.reshape(1, *data.shape) / 127.5 - 1.   # adding channel dimension and normalizing data
    epoch = 6
    
    while True:
        epoch += 1
        for batch in range(int(data.shape[-1] / m)):
            yield data[...,batch * m:(batch + 1) * m], epoch

In [7]:
class Generator(ai.Module):
    def __init__(self):
        self.g_fc = ai.Linear(z_dim, 8*gf_dim * 2 * 2)
        self.g_bn1 = ai.BatchNorm((8*gf_dim, 2, 2))
        self.g_deconv1 = ai.ConvTranspose2d(8*gf_dim, 4*gf_dim, kernel_size=5, stride=2, padding=2, a=1)
        self.g_bn2 = ai.BatchNorm((4*gf_dim, 4, 4))
        self.g_deconv2 = ai.ConvTranspose2d(4*gf_dim, 2*gf_dim, kernel_size=5, stride=2, padding=2, a=0)
        self.g_bn3 = ai.BatchNorm((2*gf_dim, 7, 7))
        self.g_deconv3 = ai.ConvTranspose2d(2*gf_dim, gf_dim, kernel_size=5, stride=2, padding=2, a=1)
        self.g_bn4 = ai.BatchNorm((gf_dim, 14, 14))
        self.g_deconv4 = ai.ConvTranspose2d(gf_dim, 1, kernel_size=5, stride=2, padding=2, a=1)
        
    def forward(self, z):
        o1 = ai.G.reshape(self.g_fc(z), (8*gf_dim, 2, 2))
        o2 = ai.G.relu(self.g_bn1(o1))
        o3 = ai.G.relu(self.g_bn2(self.g_deconv1(o2)))
        o4 = ai.G.relu(self.g_bn3(self.g_deconv2(o3)))
        o5 = ai.G.relu(self.g_bn4(self.g_deconv3(o4)))
        fake_image = ai.G.tanh(self.g_deconv4(o5))
        return fake_image

In [8]:
class Critic(ai.Module):
    def __init__(self):
        self.d_conv1 = ai.Conv2d(1, 64, kernel_size=5, stride=2, padding=2)
        self.d_conv2 = ai.Conv2d(64, 2*64, kernel_size=5, stride=2, padding=2)
        self.d_bn1 = ai.BatchNorm((2*64, 7, 7))
        self.d_conv3 = ai.Conv2d(2*64, 3*64, kernel_size=5, stride=2, padding=2)
        self.d_bn2 = ai.BatchNorm((3*64, 4, 4))
        self.d_conv4 = ai.Conv2d(3*64, 4*64, kernel_size=5, stride=2, padding=2)
        self.d_bn3 = ai.BatchNorm((4*64, 2, 2))
        self.d_fc = ai.Linear(1024, 1)
        
    def forward(self, image):
        o1 = ai.G.lrelu(self.d_conv1(image))
        o2 = ai.G.lrelu(self.d_bn1(self.d_conv2(o1)))
        o3 = ai.G.lrelu(self.d_bn2(self.d_conv3(o2)))
        o4 = ai.G.lrelu(self.d_bn3(self.d_conv4(o3)))
        o5 = self.d_fc(o4)
        return o5

In [9]:
generator = Generator()
critic = Critic()
generator.load('/content/drive/My Drive/WGAN/Generator.npy')
critic.load('/content/drive/My Drive/WGAN/Critic.npy')
print(generator)
print(critic)

loading model...
Successfully loaded model from /content/drive/My Drive/WGAN/Generator.npy
loading model...
Successfully loaded model from /content/drive/My Drive/WGAN/Critic.npy
Generator(
  g_fc: Linear(input_features=100, output_features=2048, bias=True)
  g_bn1: BatchNorm((512, 2, 2), axis=-1, momentum=0.9, bias=True)
  g_deconv1: ConvTranspose2d(512, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), a=(1, 1), bias=True)
  g_bn2: BatchNorm((256, 4, 4), axis=-1, momentum=0.9, bias=True)
  g_deconv2: ConvTranspose2d(256, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), a=(0, 0), bias=True)
  g_bn3: BatchNorm((128, 7, 7), axis=-1, momentum=0.9, bias=True)
  g_deconv3: ConvTranspose2d(128, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), a=(1, 1), bias=True)
  g_bn4: BatchNorm((64, 14, 14), axis=-1, momentum=0.9, bias=True)
  g_deconv4: ConvTranspose2d(64, 1, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), a=(1, 1), bias=True)
)
Critic(
  d_conv1: Conv2d(1, 64, ke

In [10]:
lr = 0.00005
c = 0.01
g_optim = ai.Optimizer(generator.parameters(), optim_fn='RMSProp', lr=lr)
c_optim = ai.Optimizer(critic.parameters(), optim_fn='RMSProp', lr=lr)

In [11]:
it, epoch = 0, 0
m = 64   # batch size
n_critic = 5   # number of critic updates per generator update
c = 0.01  # clip value for critic weigthts after each update

batch_size = ai.Parameter((1, 1), init_ones=True, constant=float(m), eval_grad=False)   # batch size 'm' as a Parameter constant
neg_ones = ai.Parameter((1, 1), init_ones=True, constant=-1.0, eval_grad=False)

# real images data generator
data = data_generator(m)

sample_z = np.random.uniform(-1, 1, (z_dim, m))
sampled_images = np.load('/content/drive/My Drive/WGAN/sampled_images.npy')

In [12]:
def sampler(sampled_images):
    ai.G.grad_mode = False

    # generate images like real data
    fake_images = generator.forward(sample_z).data
    fake_images = (fake_images + 1.) / 2.

    if sampled_images is not None:
        sampled_images = np.concatenate([sampled_images, fake_images], axis=-1)
    else:
        sampled_images = fake_images
    
    ai.G.grad_mode = True

    return sampled_images

In [13]:
for it in range(1201, 10000):

    # freeze generator before optimizing critic
    for p in generator.parameters():
        p.eval_grad = False

    # critic_iter = 100 if it < 25 or it%500 == 0 else n_critic

    # training critic to identify real/fake data
    for _ in range(n_critic):

        real_images, epoch = data.__next__()
        if (real_images.shape[-1] != m):
            continue

        c_loss_real = critic.forward(real_images)

        z = np.random.randn(z_dim, m)
        fake_images = generator.forward(z)

        c_loss_fake = critic.forward(fake_images)

        # add the above two losses and reduce their mean
        c_loss = c_loss_fake - c_loss_real
        c_loss = ai.G.sum(c_loss)
        c_loss = c_loss / batch_size
        c_loss.grad = np.ones(c_loss.shape)

        c_loss.backward()
        c_optim.step()
        c_optim.zero_grad()

        ai.clip_grad_value(critic.parameters(), c)

    # unfreeze generator
    for p in generator.parameters():
        p.eval_grad = True

    # training generator to fool critic with fake data
    z = np.random.uniform(-1, 1, (z_dim, m))
    fake_images = generator.forward(z)

    g_loss = critic.forward(fake_images)
    g_loss = ai.G.sum(g_loss)
    g_loss = g_loss / batch_size
    g_loss = neg_ones * g_loss
    g_loss.grad = np.ones(g_loss.shape)

    g_loss.backward()
    g_optim.step()
    g_optim.zero_grad()
    c_optim.zero_grad()

    if it%10 == 0:
        print('Iter: {}, Epoch: {}, c_loss: {}, g_loss: {}'.format(it, epoch, c_loss.data[0, 0], g_loss.data[0, 0]))
    
    if it%100 == 0:
        sampled_images=sampler(sampled_images)
        np.save('/content/drive/My Drive/WGAN/sampled_images.npy', sampled_images)
        generator.save('/content/drive/My Drive/WGAN/Generator.npy')
        critic.save('/content/drive/My Drive/WGAN/Critic.npy')

using RMSProp
using RMSProp
Iter: 1210, Epoch: 7, c_loss: -140.46321504885393, g_loss: 80.76335260953825
Iter: 1220, Epoch: 7, c_loss: -152.96775462528373, g_loss: 90.05935350143744
Iter: 1230, Epoch: 7, c_loss: -150.18990589537216, g_loss: 85.65039099019327
Iter: 1240, Epoch: 7, c_loss: -144.01648736757724, g_loss: 83.15658629748012
Iter: 1250, Epoch: 7, c_loss: -144.6526660702816, g_loss: 84.86093730710242
Iter: 1260, Epoch: 7, c_loss: -157.46680865249454, g_loss: 83.92323073367781
Iter: 1270, Epoch: 7, c_loss: -161.1495885219788, g_loss: 86.91630343738316
Iter: 1280, Epoch: 7, c_loss: -160.75946243976136, g_loss: 84.40456684299623
Iter: 1290, Epoch: 7, c_loss: -158.8976885100375, g_loss: 86.92464987170018
Iter: 1300, Epoch: 7, c_loss: -167.61122780946573, g_loss: 87.03186228243997
saving model...
Successfully saved model in /content/drive/My Drive/WGAN/Generator.npy
saving model...
Successfully saved model in /content/drive/My Drive/WGAN/Critic.npy
Iter: 1310, Epoch: 7, c_loss: -157

KeyboardInterrupt: ignored